In [59]:
import pandas as pd

In [60]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/DatasetsOxxo/MAIN/dataset_comp_ratio.csv')
df = df.dropna()
df2 = pd.read_csv('/content/drive/MyDrive/DatasetsOxxo/MAIN/dataset_comp_ratio_test.csv')
df2 = df2.dropna()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 940 entries, 0 to 948
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   TIENDA_ID                940 non-null    int64  
 1   dist_comp_directa        940 non-null    float64
 2   num_comp_directa         940 non-null    int64  
 3   dist_comp_indirecta      940 non-null    float64
 4   num_comp_indirecta       940 non-null    int64  
 5   VENTA_TOTAL              940 non-null    float64
 6   PLAZA_CVE                940 non-null    float64
 7   NIVELSOCIOECONOMICO_DES  940 non-null    object 
 8   ENTORNO_DES              940 non-null    object 
 9   MTS2VENTAS_NUM           940 non-null    float64
 10  PUERTASREFRIG_NUM        940 non-null    float64
 11  LATITUD_NUM              940 non-null    float64
 12  LONGITUD_NUM             940 non-null    float64
 13  SEGMENTO_MAESTRO_DESC    940 non-null    object 
 14  LID_UBICACION_TIENDA     940 no

# Codificar variables categoricas

In [104]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

# --- Columnas definidas ---
columnas_categoricas = [
    'ENTORNO_DES',
    #'SEGMENTO_MAESTRO_DESC',
    #'LID_UBICACION_TIENDA',
    'NIVELSOCIOECONOMICO_DES'
]

columnas_numericas = [
    'MTS2VENTAS_NUM',
    'PUERTASREFRIG_NUM',
    'dist_comp_directa',
    'num_comp_directa',
    'dist_comp_indirecta',
    'num_comp_indirecta'
]


## Pipeline preprocesamiento

In [63]:
# --- Preprocesador completo ---
preprocesador = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), columnas_categoricas),
    ('num', SimpleImputer(strategy='mean'), columnas_numericas)
])

In [64]:
X = df[columnas_categoricas + columnas_numericas]
X_pre = preprocesador.fit_transform(X)
col_names = list(preprocesador.named_transformers_['cat'].get_feature_names_out(columnas_categoricas)) + columnas_numericas
X_df = pd.DataFrame(X_pre, columns=col_names)

modelo_rf = RandomForestRegressor(random_state=42)
modelo_rf.fit(X_df, y)

importancias = pd.Series(modelo_rf.feature_importances_, index=X_df.columns).sort_values(ascending=False)
print(importancias.head(20))


MTS2VENTAS_NUM                               0.277999
dist_comp_directa                            0.202947
PUERTASREFRIG_NUM                            0.101483
dist_comp_indirecta                          0.092012
num_comp_directa                             0.069267
NIVELSOCIOECONOMICO_DES_AB                   0.043341
NIVELSOCIOECONOMICO_DES_C                    0.022396
NIVELSOCIOECONOMICO_DES_BC                   0.022260
num_comp_indirecta                           0.019774
ENTORNO_DES_Receso                           0.017710
LID_UBICACION_TIENDA_UT_CARRETERA_GAS        0.016902
LID_UBICACION_TIENDA_UT_DENSIDAD             0.015267
ENTORNO_DES_Hogar                            0.014499
LID_UBICACION_TIENDA_UT_TRAFICO_VEHICULAR    0.014310
ENTORNO_DES_Base                             0.013861
NIVELSOCIOECONOMICO_DES_A                    0.011435
LID_UBICACION_TIENDA_UT_GAS_URBANA           0.011072
NIVELSOCIOECONOMICO_DES_B                    0.010694
NIVELSOCIOECONOMICO_DES_CD  

# Modelos de regresion

In [71]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
#import decision tree regressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 1. Definir X e y para entrenamiento y prueba
X_train = df[columnas_categoricas + columnas_numericas]
y_train = df['RATIO_CUMPLIMIENTO']

X_test = df2[columnas_categoricas + columnas_numericas]
y_test = df2['RATIO_CUMPLIMIENTO']

# 2. Crear Pipeline base
pipeline = Pipeline([
    ('preprocesador', preprocesador),
    ('regresor', RandomForestRegressor(random_state=42))
])

# 3. Definir espacio de búsqueda de hiperparámetros
param_grid = {
    'regresor__n_estimators': [100, 200],
    'regresor__max_depth': [None, 10, 20],
    'regresor__min_samples_split': [2, 5],
    'regresor__min_samples_leaf': [1, 2, 4]
}

# 4. Configurar GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=6,  # validación cruzada
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=2
)

# 5. Entrenar búsqueda
grid_search.fit(X_train, y_train)

# 6. Evaluar en datos de prueba
y_pred = grid_search.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# 7. Mostrar resultados
print("📊 Métricas del mejor modelo en datos de prueba:")
print(f"MAE  : {mae:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"R²   : {r2:.4f}")
print("🏆 Mejor combinación de hiperparámetros:")
print(grid_search.best_params_)


Fitting 6 folds for each of 36 candidates, totalling 216 fits


KeyboardInterrupt: 

In [80]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 1. Separar X e y para entrenamiento y prueba
X_train = df[columnas_categoricas + columnas_numericas]
y_train = df['RATIO_CUMPLIMIENTO']

X_test = df2[columnas_categoricas + columnas_numericas]
y_test = df2['RATIO_CUMPLIMIENTO']

# 2. Crear pipeline con modelo
modelo = Pipeline([
    ('preprocesador', preprocesador),
    ('regresor', DecisionTreeRegressor(random_state=42, max_depth=6, min_samples_leaf=4))
])

# 3. Entrenar el modelo
modelo.fit(X_train, y_train)

# 4. Predecir sobre datos de prueba
y_pred = modelo.predict(X_test)

# 5. Calcular métricas
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# 6. Mostrar resultados
print("📊 Métricas del modelo en datos de prueba:")
print(f"MAE  (Error Absoluto Medio): {mae:.4f}")
print(f"RMSE (Raíz del Error Cuadrático Medio): {rmse:.4f}")
print(f"R²   (Coeficiente de Determinación): {r2:.4f}")


📊 Métricas del modelo en datos de prueba:
MAE  (Error Absoluto Medio): 0.6652
RMSE (Raíz del Error Cuadrático Medio): 0.8292
R²   (Coeficiente de Determinación): -0.0710


In [67]:
import warnings
#importar cros val scores
from sklearn.model_selection import cross_val_score
warnings.filterwarnings('ignore')

feature_cols = [
    'NIVELSOCIOECONOMICO_DES',
    'ENTORNO_DES',
    'MTS2VENTAS_NUM',
    'PUERTASREFRIG_NUM',
    'SEGMENTO_MAESTRO_DESC',
    'LID_UBICACION_TIENDA',
    'dist_comp_directa',
    'num_comp_directa',
    'dist_comp_indirecta',
    'num_comp_indirecta',
]

columnas_excluir = [
    'RATIO_CUMPLIMIENTO',
    'VENTA_TOTAL',
    'TIENDA_ID',
    'PLAZA_CVE',
    'DATASET',
    'META_VENTA',
]

X = df.drop(columns=columnas_excluir)

y = df['RATIO_CUMPLIMIENTO']

# Entrenar preprocesador solo
preprocesador.fit(X)

# Transformar X
X_transformado = preprocesador.transform(X)

# Obtener nombres de columnas transformadas
cat_features = preprocesador.named_transformers_['cat'].get_feature_names_out(columnas_categoricas)
nuevos_nombres = list(cat_features) + columnas_numericas

X_df = pd.DataFrame(X_transformado, columns=nuevos_nombres)

from sklearn.ensemble import RandomForestRegressor

modelo_rf = RandomForestRegressor(random_state=42)
modelo_rf.fit(X_df, y)

importancias = pd.Series(modelo_rf.feature_importances_, index=X_df.columns)
importancias = importancias.sort_values(ascending=False)

print("Top características más importantes:")
print(importancias.head(20))


from sklearn.ensemble import RandomForestRegressor

#seleccion caracteristicas
modelo_rf = RandomForestRegressor(random_state=42)
modelo_rf.fit(X, y)

# 4. Importancia de cada variable
importancias = pd.Series(modelo_rf.feature_importances_, index=X.columns)
importancias = importancias.sort_values(ascending=False)

# 5. Mostrar las más importantes
print("Top características más importantes:")
print(importancias.head(20))

Top características más importantes:
MTS2VENTAS_NUM                               0.277999
dist_comp_directa                            0.202947
PUERTASREFRIG_NUM                            0.101483
dist_comp_indirecta                          0.092012
num_comp_directa                             0.069267
NIVELSOCIOECONOMICO_DES_AB                   0.043341
NIVELSOCIOECONOMICO_DES_C                    0.022396
NIVELSOCIOECONOMICO_DES_BC                   0.022260
num_comp_indirecta                           0.019774
ENTORNO_DES_Receso                           0.017710
LID_UBICACION_TIENDA_UT_CARRETERA_GAS        0.016902
LID_UBICACION_TIENDA_UT_DENSIDAD             0.015267
ENTORNO_DES_Hogar                            0.014499
LID_UBICACION_TIENDA_UT_TRAFICO_VEHICULAR    0.014310
ENTORNO_DES_Base                             0.013861
NIVELSOCIOECONOMICO_DES_A                    0.011435
LID_UBICACION_TIENDA_UT_GAS_URBANA           0.011072
NIVELSOCIOECONOMICO_DES_B                    

ValueError: could not convert string to float: 'BC'

# Modelos clasificacion

In [146]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

# Crear target categórico
def clasificar_ratio(r):
    if r < 1.3:
        return 'Bajo'
    elif r < 1.6:
        return 'Medio'
    elif r < 2.7:
        return 'Alto'
    else:
        return 'Genial'

df['CLASE_CUMPLIMIENTO'] = df['RATIO_CUMPLIMIENTO'].apply(clasificar_ratio)
df2['CLASE_CUMPLIMIENTO'] = df2['RATIO_CUMPLIMIENTO'].apply(clasificar_ratio)

In [147]:
print("📊 Proporción de clases en df (entrenamiento):")
print(df['CLASE_CUMPLIMIENTO'].value_counts(normalize=True).round(3))

print("\n📊 Proporción de clases en df2 (prueba):")
print(df2['CLASE_CUMPLIMIENTO'].value_counts(normalize=True).round(3))


📊 Proporción de clases en df (entrenamiento):
CLASE_CUMPLIMIENTO
Alto      0.524
Genial    0.229
Medio     0.132
Bajo      0.115
Name: proportion, dtype: float64

📊 Proporción de clases en df2 (prueba):
CLASE_CUMPLIMIENTO
Alto      0.495
Genial    0.248
Medio     0.143
Bajo      0.114
Name: proportion, dtype: float64


## Random Forest features num y cat

In [148]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

feature_cols = [
    'NIVELSOCIOECONOMICO_DES',
    'ENTORNO_DES',
    'MTS2VENTAS_NUM',
    'PUERTASREFRIG_NUM',
    'SEGMENTO_MAESTRO_DESC',
    'LID_UBICACION_TIENDA',
    'dist_comp_directa',
    'num_comp_directa',
    'dist_comp_indirecta',
    'num_comp_indirecta',
]

# --- Features y target categórico ---
X_train = df[feature_cols]
y_train = df['CLASE_CUMPLIMIENTO']

X_test = df2[feature_cols]
y_test = df2['CLASE_CUMPLIMIENTO']

# --- Pipeline con preprocesamiento y clasificador ---
pipeline = Pipeline([
    ('preprocesador', preprocesador),
    ('clasificador', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# --- Hiperparámetros para búsqueda ---
param_grid = {
    'clasificador__n_estimators': [100, 200],
    'clasificador__max_depth': [None, 10, 20],
    'clasificador__min_samples_split': [2, 5],
    'clasificador__min_samples_leaf': [1, 2]
}

# --- GridSearchCV ---
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='f1_macro',  # mide el desempeño promedio entre clases desbalanceadas
    n_jobs=-1,
    verbose=2
)

# --- Entrenamiento del GridSearch ---
grid_search.fit(X_train, y_train)

# --- Predicción y evaluación en df2 ---
y_pred = grid_search.predict(X_test)

print("📊 Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

print("📉 Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

print("🏆 Mejor combinación de hiperparámetros:")
print(grid_search.best_params_)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
📊 Reporte de Clasificación:
              precision    recall  f1-score   support

        Alto       0.56      0.69      0.62        52
        Bajo       0.17      0.08      0.11        12
      Genial       0.53      0.38      0.44        26
       Medio       0.50      0.53      0.52        15

    accuracy                           0.52       105
   macro avg       0.44      0.42      0.42       105
weighted avg       0.50      0.52      0.50       105

📉 Matriz de Confusión:
[[36  3  6  7]
 [ 9  1  2  0]
 [14  1 10  1]
 [ 5  1  1  8]]
🏆 Mejor combinación de hiperparámetros:
{'clasificador__max_depth': 10, 'clasificador__min_samples_leaf': 2, 'clasificador__min_samples_split': 5, 'clasificador__n_estimators': 200}


In [149]:
# 1. Obtener los nombres originales de variables numéricas (no cambian)
nombres_numericos = columnas_numericas

# 2. Obtener nombres expandidos de columnas categóricas (OneHotEncoder)
ohe = grid_search.best_estimator_.named_steps['preprocesador'].named_transformers_['cat']
nombres_categoricos = ohe.get_feature_names_out(columnas_categoricas)

# 3. Unir todos los nombres
nombres_finales = list(nombres_numericos) + list(nombres_categoricos)

# 4. Obtener las importancias del modelo (si es un modelo de árbol)
importancias = grid_search.best_estimator_.named_steps['clasificador'].feature_importances_

# 5. Crear un DataFrame para visualizarlas
importancia_df = pd.DataFrame({
    'feature': nombres_finales,
    'importancia': importancias
}).sort_values(by='importancia', ascending=False)

# 6. Ver top N
print("🎯 Top 20 features más importantes:")
print(importancia_df.head(20))


🎯 Top 20 features más importantes:
                       feature  importancia
0               MTS2VENTAS_NUM     0.249833
1            dist_comp_directa     0.182702
2          dist_comp_indirecta     0.130108
5            PUERTASREFRIG_NUM     0.108546
3             num_comp_directa     0.098911
4           num_comp_indirecta     0.050667
14   NIVELSOCIOECONOMICO_DES_C     0.032470
7            ENTORNO_DES_Hogar     0.029497
9           ENTORNO_DES_Receso     0.022179
6             ENTORNO_DES_Base     0.022175
11  NIVELSOCIOECONOMICO_DES_AB     0.017571
13  NIVELSOCIOECONOMICO_DES_BC     0.016940
12   NIVELSOCIOECONOMICO_DES_B     0.016626
15  NIVELSOCIOECONOMICO_DES_CD     0.012774
16   NIVELSOCIOECONOMICO_DES_D     0.005335
10   NIVELSOCIOECONOMICO_DES_A     0.002450
8         ENTORNO_DES_Peatonal     0.001219


## Random forest segm. entorno


In [151]:
from sklearn.metrics import classification_report, confusion_matrix

# Diccionario para guardar resultados por entorno
resultados_por_entorno = {}

# Recorremos cada entorno único
for entorno in df['ENTORNO_DES'].unique():
    print(f"\n🔍 Entrenando modelo para ENTORNO: {entorno}")

    # Filtrar datos por entorno
    df_train_e = df[df['ENTORNO_DES'] == entorno].copy()
    df_test_e = df2[df2['ENTORNO_DES'] == entorno].copy()

    # Verificar si hay suficientes datos
    if len(df_train_e) < 30 or len(df_test_e) < 10:
        print(f"⚠️ Muy pocos datos para el entorno '{entorno}' — se omite.")
        continue

    # Preparar features y target
    X_train = df_train_e[feature_cols]
    y_train = df_train_e['CLASE_CUMPLIMIENTO']

    X_test = df_test_e[feature_cols]
    y_test = df_test_e['CLASE_CUMPLIMIENTO']

    # Re-crear el GridSearch (para reiniciarlo limpio por entorno)
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=5,
        scoring='f1_macro',
        n_jobs=-1,
        verbose=0
    )

    # Entrenar modelo
    grid_search.fit(X_train, y_train)

    # Predecir
    y_pred = grid_search.predict(X_test)

    # Reporte
    reporte = classification_report(y_test, y_pred, output_dict=True)
    matriz = confusion_matrix(y_test, y_pred)

    resultados_por_entorno[entorno] = {
        "reporte": reporte,
        "confusion_matrix": matriz,
        "accuracy": reporte["accuracy"],
        "mejores_hiperparametros": grid_search.best_params_
    }

    print(f"✅ Accuracy: {reporte['accuracy']:.3f}")
    print(f"🏆 Mejores hiperparámetros: {grid_search.best_params_}")



🔍 Entrenando modelo para ENTORNO: Hogar
✅ Accuracy: 0.412
🏆 Mejores hiperparámetros: {'clasificador__max_depth': 10, 'clasificador__min_samples_leaf': 2, 'clasificador__min_samples_split': 5, 'clasificador__n_estimators': 100}

🔍 Entrenando modelo para ENTORNO: Base
✅ Accuracy: 0.526
🏆 Mejores hiperparámetros: {'clasificador__max_depth': 10, 'clasificador__min_samples_leaf': 1, 'clasificador__min_samples_split': 5, 'clasificador__n_estimators': 100}

🔍 Entrenando modelo para ENTORNO: Receso
✅ Accuracy: 0.538
🏆 Mejores hiperparámetros: {'clasificador__max_depth': None, 'clasificador__min_samples_leaf': 1, 'clasificador__min_samples_split': 2, 'clasificador__n_estimators': 100}

🔍 Entrenando modelo para ENTORNO: Peatonal
⚠️ Muy pocos datos para el entorno 'Peatonal' — se omite.


## Random Forest feature num

In [115]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# --- Variables seleccionadas ---
features_top = [
    'MTS2VENTAS_NUM',
    'PUERTASREFRIG_NUM',
    'dist_comp_directa',
    'num_comp_directa',
    'dist_comp_indirecta',
    'num_comp_indirecta'
]

# --- Separar X e y ---
X_train = df[features_top]
y_train = df['CLASE_CUMPLIMIENTO']

X_test = df2[features_top]
y_test = df2['CLASE_CUMPLIMIENTO']

# --- Pipeline (solo imputación si aplica, sin codificación categórica) ---
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Todas son numéricas
preprocesador = ColumnTransformer(transformers=[
    ('num', SimpleImputer(strategy='mean'), features_top)
])

pipeline = Pipeline([
    ('preprocesador', preprocesador),
    ('clasificador', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# --- Hiperparámetros a buscar ---
param_grid = {
    'clasificador__n_estimators': [100, 200],
    'clasificador__max_depth': [None, 10, 20],
    'clasificador__min_samples_split': [2, 5],
    'clasificador__min_samples_leaf': [1, 2]
}

# --- GridSearchCV ---
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=2
)

# --- Entrenar modelo ---
grid_search.fit(X_train, y_train)

# --- Evaluación en test ---
y_pred = grid_search.predict(X_test)

print("📊 Reporte de Clasificación (solo top features):")
print(classification_report(y_test, y_pred))
print("📉 Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("🏆 Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
📊 Reporte de Clasificación (solo top features):
              precision    recall  f1-score   support

        Alto       0.45      0.54      0.49        37
        Bajo       0.00      0.00      0.00         3
      Genial       0.41      0.37      0.39        38
       Medio       0.35      0.33      0.34        27

    accuracy                           0.41       105
   macro avg       0.30      0.31      0.31       105
weighted avg       0.40      0.41      0.40       105

📉 Matriz de Confusión:
[[20  1 10  6]
 [ 1  0  0  2]
 [15  0 14  9]
 [ 8  0 10  9]]
🏆 Mejores hiperparámetros encontrados:
{'clasificador__max_depth': 10, 'clasificador__min_samples_leaf': 2, 'clasificador__min_samples_split': 5, 'clasificador__n_estimators': 100}


## XGBoost features num

In [118]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import classification_report, confusion_matrix

# --- Variables importantes ---
features_top = [
    'MTS2VENTAS_NUM',
    'PUERTASREFRIG_NUM',
    'dist_comp_directa',
    'num_comp_directa',
    'dist_comp_indirecta',
    'num_comp_indirecta'
]

# --- Datos de entrenamiento y prueba ---
X_train = df[features_top]
y_train = df['CLASE_CUMPLIMIENTO']

X_test = df2[features_top]
y_test = df2['CLASE_CUMPLIMIENTO']

# --- Preprocesador (solo imputación numérica) ---
preprocesador = ColumnTransformer(transformers=[
    ('num', SimpleImputer(strategy='mean'), features_top)
])

# --- Pipeline con XGBoost ---
pipeline = Pipeline([
    ('preprocesador', preprocesador),
    ('clasificador', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'))
])
# --- Codificar etiquetas ---

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df['CLASE_CUMPLIMIENTO'])
y_test = label_encoder.transform(df2['CLASE_CUMPLIMIENTO'])

# --- GridSearch de hiperparámetros ---
param_grid = {
    'clasificador__n_estimators': [100, 200],
    'clasificador__max_depth': [3, 6, 10],
    'clasificador__learning_rate': [0.01, 0.1, 0.3],
    'clasificador__subsample': [0.8, 1],
    'clasificador__colsample_bytree': [0.8, 1]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring='f1_macro',  # buena para clases desbalanceadas
    verbose=2,
    n_jobs=-1
)

# --- Entrenar modelo ---
grid_search.fit(X_train, y_train)

# --- Evaluar en conjunto de prueba ---
y_pred = grid_search.predict(X_test)

print("📊 Reporte de Clasificación con XGBoost:")
print(classification_report(y_test, y_pred))
print("📉 Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("🏆 Mejores hiperparámetros:")
print(grid_search.best_params_)


Fitting 3 folds for each of 72 candidates, totalling 216 fits
📊 Reporte de Clasificación con XGBoost:
              precision    recall  f1-score   support

           0       0.35      0.49      0.41        37
           1       0.00      0.00      0.00         3
           2       0.33      0.32      0.32        38
           3       0.22      0.15      0.18        27

    accuracy                           0.32       105
   macro avg       0.23      0.24      0.23       105
weighted avg       0.30      0.32      0.31       105

📉 Matriz de Confusión:
[[18  0 12  7]
 [ 2  0  0  1]
 [20  0 12  6]
 [11  0 12  4]]
🏆 Mejores hiperparámetros:
{'clasificador__colsample_bytree': 1, 'clasificador__learning_rate': 0.3, 'clasificador__max_depth': 3, 'clasificador__n_estimators': 100, 'clasificador__subsample': 1}


## XGBoost feature num y cat

In [145]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# --- Variables ---
features_top = [
    'MTS2VENTAS_NUM',
    'dist_comp_directa',
    'dist_comp_indirecta',
    'num_comp_directa',
    'num_comp_indirecta',
    'PUERTASREFRIG_NUM'
]

columnas_categoricas = [
    'ENTORNO_DES',
    'NIVELSOCIOECONOMICO_DES'
]

columnas_numericas = features_top  # Todas tus numéricas clave

features_finales = columnas_numericas + columnas_categoricas

# --- Separar datos ---
X_train = df[features_finales]
y_train = df['CLASE_CUMPLIMIENTO']

X_test = df2[features_finales]
y_test = df2['CLASE_CUMPLIMIENTO']

# --- Codificar y transformar etiquetas ---
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_test_enc = label_encoder.transform(y_test)

# --- Preprocesador con OneHotEncoder para categóricas ---
preprocesador = ColumnTransformer(transformers=[
    ('num', SimpleImputer(strategy='mean'), columnas_numericas),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), columnas_categoricas)
])

# --- Pipeline con XGBoost ---
pipeline = Pipeline([
    ('preprocesador', preprocesador),
    ('clasificador', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'))
])

# --- Hiperparámetros ---
param_grid = {
    'clasificador__n_estimators': [100, 200],
    'clasificador__max_depth': [3, 6, 10],
    'clasificador__learning_rate': [0.01, 0.1, 0.3],
    'clasificador__subsample': [0.8, 1],
    'clasificador__colsample_bytree': [0.8, 1]
}

# --- GridSearchCV ---
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,
    scoring='f1_macro',
    verbose=2,
    n_jobs=-1
)

# --- Entrenar ---
grid_search.fit(X_train, y_train_enc)

# --- Predecir y evaluar ---
y_pred_enc = grid_search.predict(X_test)
y_pred = label_encoder.inverse_transform(y_pred_enc)
y_test_labels = label_encoder.inverse_transform(y_test_enc)

print("\n📊 Reporte de Clasificación con XGBoost + OneHot:")
print(classification_report(y_test_labels, y_pred))

print("\n📉 Matriz de Confusión:")
print(confusion_matrix(y_test_labels, y_pred))

print("\n🏆 Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)


Fitting 3 folds for each of 72 candidates, totalling 216 fits

📊 Reporte de Clasificación con XGBoost + OneHot:
              precision    recall  f1-score   support

        Alto       0.52      0.79      0.63        52
        Bajo       0.33      0.08      0.13        12
      Genial       0.39      0.27      0.32        26
       Medio       0.40      0.13      0.20        15

    accuracy                           0.49       105
   macro avg       0.41      0.32      0.32       105
weighted avg       0.45      0.49      0.43       105


📉 Matriz de Confusión:
[[41  1  8  2]
 [10  1  1  0]
 [18  0  7  1]
 [10  1  2  2]]

🏆 Mejores hiperparámetros encontrados:
{'clasificador__colsample_bytree': 0.8, 'clasificador__learning_rate': 0.3, 'clasificador__max_depth': 3, 'clasificador__n_estimators': 200, 'clasificador__subsample': 0.8}
